## Newtonian Elliptical Orbit
This model aims to plot the orbit of two black holes with no post-newtonian corrections. Thus, it is a typical elliptical orbit of two masses $M_1$ and $M_2$. We implement the RK4 method in the model.

In [1]:
# imports
import numpy as np
import math
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [2]:
# gravitational constant
G = 6.67430e-11


In [18]:
# acceleration and velocity ODEs
# in x-y plane, there are 8 total terms (2 for velocity, 2 for acceleration x2 for each body)
# the parameter state = [x, y, vx, vy]
def ODEs(states, m1, m2):
    # define states
    state1, state2 = states[0], states[1]

    # define positions
    x1, x2 = state1[0], state2[0]
    y1, y2 = state1[1], state2[1]

    # velocities
    vx1, vx2 = state1[2], state2[2]
    vy1, vy2 = state1[3], state2[3]

    # separation distance
    r = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    r3 = r ** 3
    assert r3 != 0, "Cannot divide by zero"

    # ODEs: mass 1
    dxdt_m1 = vx1
    dydt_m1 = vy1

    dx2dt2_m1 = (-G * m2) / r3
    dy2dt2_m1 = (-G * m2) / r3

    # ODEs: mass 2
    dxdt_m2 = vx2
    dydt_m2 = vy2

    dx2dt2_m2 = (-G * m1) / r3
    dy2dt2_m2 = (-G * m1) / r3

    # return derivatives
    s1 = np.array([dxdt_m1, dydt_m1, dx2dt2_m1, dy2dt2_m1], float)
    s2 = np.array([dxdt_m2, dydt_m2, dx2dt2_m2, dy2dt2_m2], float)
    return np.vstack((s1, s2))

In [19]:
# RK4 method
def rk4(s, m1, m2, dt):
    k1 = dt * ODEs(s, m1, m2)
    k2 = dt * ODEs(s + 0.5 * k1, m1, m2)
    k3 = dt * ODEs(s + 0.5 * k2, m1, m2)
    k4 = dt * ODEs(s + k3, m1, m2)

    correction = (1/6) * (k1 + (2 * k2) + (2*k3) + k4)
    return s + correction

In [35]:
# simple class to hold the body's paramaters and methods
class Body(object):
    def __init__(self, mass, initial_x, initial_y, initial_vx, initial_vy):
        self.mass = mass
        self.initial_x = initial_x
        self.initial_y = initial_y
        self.initial_vx = initial_vx
        self.initial_vy = initial_vy
        self.state = np.array([self.initial_x, self.initial_y, self.initial_vx, self.initial_vy])

    def set_zeros(size):
        self.x = np.zeros(size)
        self.y = np.zeros(size)
        self.vx = np.zeros(size)
        self.vy = np.zeros(size)

        # set initial state in array
        self.x[0] = self.initial_x
        self.y[0] = self.initial_y
        self.vx[0] = self.initial_vx
        self.vy[0] = self.initial_vy

        return self.x, self.y, self.vx, self.vy


    def get_state(i):
        return np.array([self.x[i]], self.y[i], self.vx[i], self.vy[i])
    
    def set_state(i, s):
        self.x[i] = s[0]
        self.y[i] = s[1]
        self.vx[i] = s[2]
        self.vy[i] = s[3]

    
        

In [36]:
def run_orbit(body1, body2, start, end, N):
    dt = (end - start) / N
    # set zeros for both masses
    body1.set_zeros(N)
    body2.set_zeros(N)

    # run rk4 method
    for i in range(1, N):
        # get states for each mass
        s1 = body1.get_state(i-1)
        s2 = body2.get_state(i-1)

        s_prev = np.vstack((s1, s2))

        # run rk4
        s_new = rk4(s, body1.mass, body2.mass, dt)

        s1_new = s_new[0]
        s2_new = s_new[1]

        # set the state
        body1.set_state(i, s1_new)
        body2.set_state(i, s2_new)
    

In [ ]:
# Example usage
# define black holes
b1 = 